# Computing Optimal Strategies for Naughts & Crosses via Brute Force

The problem is to 'play' all possible games in advance and store the results, such that when a player is presented with any state of the game they can choose as their next move the one with highest probability of leading to a win. Playing all possible games in this way is a 'brute force' approach to learning strategies. This algorithm will require:

1. A way to represent the state of a game that can also serve as an index to query against;
2. A method for identifying a winning state;
2. A method for computing the set of all possible moves conditional on the current state of the game;
3. A method for computing the probability of winning conditional on the state of the game; and,
4. An algorithm for playing all possible games and computing (2) and (3).

## Imports


In [1]:
# imports will go here

## Representing Game State

There are 9 cells in which a move can be made (the board is a 3 x 3 grid), and each cell can be in 1 of 3 possible states:

1. Player One has made a move in the cell (`"1"`);
2. Player Two has made a move in the cell (`"2"`); or,
3. No player has made a move in the cell (`"0"`).

In everything that follows Player One will always start and we will view the same entire from within their frame-of-reference (as the game is symmetrical between players) - i.e., we will assume that we are always Player One as far as computing win probabilities are concerned.

A viable way of representing state is as a 9 digit integer comprised solely of 3 values - e.g., the 3 x 3 (matrix) representation of the board

```text
102
012
100
```

Could just as easily be represented in a 'vector-like' setup,

```text
102012100
```

This can also be encoded as a string (to enable easy element lookup via its list protocol), that can also be used as an index to search against - e.g., as a dictionary key.

In [6]:
game_states = {
    "102012100": {
        "possible_moves": {
            "112012100": {"p_win": 0.01},
            "102112100": {"p_win": 0.02},
            "102012110": {"p_win": 0.03},
            "102012101": {"p_win": 0.04},
        }
    }
}

next_move_options = game_states["102012100"]["possible_moves"]
next_move_options

{'112012100': {'p_win': 0.01},
 '102112100': {'p_win': 0.02},
 '102012110': {'p_win': 0.03},
 '102012101': {'p_win': 0.04}}

## Identifying Winners

There are only 8 possible states that map to a win - 3 from the columns, 3 rom the rows, and 2 from the diagonals. We can check the state of the game explicitly against these.

In [21]:
def winning_state(game_state: str, player: str = "1") -> bool:
    """Check the game state to see if it indicates a winning state."""
    # check rows
    if game_state[0] == game_state[1] == game_state[2] == player:
        return True
    if game_state[3] == game_state[4] == game_state[5] == player:
        return True
    if game_state[6] == game_state[7] == game_state[8] == player:
        return True

    # check columns
    if game_state[0] == game_state[3] == game_state[6] == player:
        return True
    if game_state[1] == game_state[4] == game_state[7] == player:
        return True
    if game_state[2] == game_state[5] == game_state[8] == player:
        return True

    # check diagonals
    if game_state[0] == game_state[4] == game_state[8] == player:
        return True
    if game_state[6] == game_state[4] == game_state[8] == player:
        return True

    # otherwise...
    return False


print(f"111212122: win for Player One --> {winning_state('111212122')}")
print(f"120120100: win for Player One --> {winning_state('111212100')}")
print(f"120210001: win for Player One --> {winning_state('120210001')}")
print(f"201210100: win for Player One --> {winning_state('120210001')}")
print(f"112012100: win for Player One --> {winning_state('112012100')}")

111212122: win for Player One --> True
120120100: win for Player One --> True
120210001: win for Player One --> True
201210100: win for Player One --> True
112012100: win for Player One --> False


## Identifying Possible Moves